Generate material maps using an existing model checkpoint.

In [1]:
import geopandas as gpd
import lightning.pytorch
import matplotlib.pyplot as plt
import numpy as np
import rasterio.features
import rasterio.plot
import torch.utils.data
import torchgeo.datasets
import torchgeo.samplers
import tqdm

import training.datamodule
import training.dataset
import training.task

### Setup

In [2]:
# Enable TF32 support.
torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

# Fix the RNGs.
lightning.pytorch.seed_everything(42, workers=True)

Seed set to 42


42

### Testing

In [6]:
# Instantiate the model in inference mode.
model = training.task.TrainingTask.load_from_checkpoint("../logs/RoofSense/best-v0.ckpt")
# NOTE: This command disables all batch normalization and dropout layers.
model.eval()

# Instantiate the datamodule.
datamodule = training.datamodule.TrainingDataModule(root="../dataset/temp",
                                                           batch_size=16,
                                                           num_workers=8)

# Instantiate the trainer.
trainer = lightning.Trainer(benchmark=True)

weights


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
# Test the model.
# FIXME: Do not log the test results.
trainer.test(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      tst/Accuracy/1       │   0.029239006340503693    │
│      tst/Accuracy/2       │            0.0            │
│      tst/Accuracy/3       │            0.0            │
│      tst/Accuracy/4       │    0.9304760098457336     │
│      tst/Accuracy/5       │    0.9928029775619507     │
│      tst/Accuracy/6       │     0.630875825881958     │
│      tst/Accuracy/7       │    0.9905111789703369     │
│      tst/Accuracy/8       │            0.0            │
│       tst/F1Score/1       │    0.05625990405678749    │
│       tst/F1Score/2       │            0.0            │
│       tst/F1Score/3       │            0.0            │
│       tst/F1Score/4       │    0.7835902571678162     │
│       tst/F1Score/5       │    0.9937304854393005     │
│       tst/F1Score/6       │    0.7058420777320862     │
│       tst/F1Score/7       │    0.9891720414161682     │
│       tst/F1Score/8       │            0.0            │
│     tst/MacroAccuracy     │     0.397100567817688     │
│     tst/MacroF1Score      │     0.392066091299057     │
│     tst/MicroAccuracy     │    0.7430708408355713     │
│     tst/MicroF1Score      │    0.7430708408355713     │
│         tst/loss          │    1.6668930053710938     │
└───────────────────────────┴───────────────────────────┘

[{'tst/loss': 1.6668930053710938,
  'tst/MacroAccuracy': 0.397100567817688,
  'tst/MacroF1Score': 0.392066091299057,
  'tst/MicroAccuracy': 0.7430708408355713,
  'tst/MicroF1Score': 0.7430708408355713,
  'tst/Accuracy/1': 0.029239006340503693,
  'tst/Accuracy/2': 0.0,
  'tst/Accuracy/3': 0.0,
  'tst/Accuracy/4': 0.9304760098457336,
  'tst/Accuracy/5': 0.9928029775619507,
  'tst/Accuracy/6': 0.630875825881958,
  'tst/Accuracy/7': 0.9905111789703369,
  'tst/Accuracy/8': 0.0,
  'tst/F1Score/1': 0.05625990405678749,
  'tst/F1Score/2': 0.0,
  'tst/F1Score/3': 0.0,
  'tst/F1Score/4': 0.7835902571678162,
  'tst/F1Score/5': 0.9937304854393005,
  'tst/F1Score/6': 0.7058420777320862,
  'tst/F1Score/7': 0.9891720414161682,
  'tst/F1Score/8': 0.0}]